In [4]:
import json
import urllib2
import urllib
import xmltodict
import os
import string
from ebooklib import epub 
from  comFunc import *
from multiprocessing import Pool
import time
import re
import functools

In [59]:

def movet(titles):
    #print titles
    metakey='http://purl.org/dc/elements/1.1/'
    f=titles[1][0]+"/"+titles[0]+".epub"
    ti=titles[1][1][u'title'].encode('ascii','ignore').replace("&"," and ")
    
    tmt=''.join(tt for tt in ti if tt  in string.ascii_letters+string.digits+" &()")
    series=''
    index=-1
    if tmt.count('(')>0:
        title=tmt[:tmt.index('(')].strip()
        seri=tmt[tmt.index('(')+1:tmt.index(")")].split()
        try:
            index=float(seri[-1])
            
            series=' '.join(seri[:-1]).strip()
        except:
            series=' '.join(seri).strip()
    else:
        title=''.join(tt for tt in ti.encode('ascii','ignore') if tt  in string.ascii_letters+string.digits+" '.,-&")
    #print index 
    au=''.join (t for t in titles[1][1][u'authors'][0].encode('ascii','ignore').strip() if t in string.ascii_letters+" ." )
    if len(au.split())<2:
        return "A",f
    auth=' '.join(au.split())
    title=title.replace ("By "+au,'').strip()
    title=' '.join(m.strip() for m in title.split())
    if title.count(':')>0:
        title=title.split(':')[0].strip()
    tt="/media/sf_lbuntu/books/"+au.strip()+" - "+title.strip()+".epub"
    print ti,au,auth
    tmp="/tmp/"+auth.strip()+" - "+title.strip()+".epub"
    try:
        book=epub.read_epub(f)
        
    except:
        return "R",f
    book.metadata
    #if 'creator' in book.metadata[metakey].keys():
    book.set_unique_metadata( namespace, name, value, others=None)
        if u'authors' in titles[1][1].keys():
            book.metadata[metakey].pop('creator')
            for ar in titles[1][1][u'authors']:
                book.add_author(ar.encode('ascii','ignore'))
        
    #if 'identifier' in book.metadata[metakey].keys():
        book.metadata[metakey].pop('identifier')
        if u'industryIdentifiers' in titles[1][1].keys():
            for idf in titles[1][1][u'industryIdentifiers']:
                if u'identifier' in idf.keys():
                    book.set_identifier(idf[u'identifier'])
    #if 'publisher' in book.metadata[metakey].keys():
        book.metadata[metakey].pop('publisher')
        #book.metadata[metakey]['publisher']=titles[1][1][u'publisher']
    #if 'date' in book.metadata[metakey].keys():
        book.metadata[metakey].pop('date')

    
    print book.metadata[metakey].keys()
    print titles[1][1].keys()
    #book.metadata[metakey]=titles[1][1]
    if not os.path.isfile(tt):
        #try:
        epub.write_epub(tmp,book,{} )
        #except:
        #return "W",f,tmp
            

        if os.path.exists(tmp):

            #!ebook-meta "$tmp" --title "$ti" --authors "$auth" --language "eng" >>/dev/null 2>>/dev/null

            #if index>0 and len(series)>0: 
             #   !ebook-meta "$tmp" --series "$series" --index "$index" >>/dev/null 2>>/dev/null

            !mv "$tmp" "$tt"
            return "S",f
        else:
            return "F",f
    else:
            return "E",f
#E=Exists, F=failed, S=success, W=Writing error, R=reading error, A=Author problem

In [47]:
titles=load_obj('data/titlesISBN')#{}
#unpacktitles
uptit=[[k,titles[k]] for k in titles.keys()]
prockeys=[]#load_obj('data/prockeysIm')
flist=list(filter(lambda x: x[0] not in prockeys and type(x[1][1])==dict, uptit))

len(flist),len(titles.keys()),len(prockeys)

(7, 8, 0)

In [60]:
movet(flist[-1])

A.S. Byatt Mariadele Boccardi Mariadele Boccardi
['description', 'language', 'title', 'creator', 'contributor', 'identifier', 'subject']
[u'publisher', u'description', u'language', u'previewLink', u'readingModes', u'allowAnonLogging', u'publishedDate', u'printType', u'pageCount', u'maturityRating', u'contentVersion', u'industryIdentifiers', u'authors', u'title', u'imageLinks', u'infoLink', u'categories', u'canonicalVolumeLink']


('S',
 "/media/some/sophie-hhd/transmitt/fic/#Calibre Fiction/A. S. Byatt/Djinn in the Nightingale's Eye (2440)/Djinn in the Nightingale's Eye - A. S. Byatt.epub")

In [40]:
flist[1][1][1][u'categories']

[u'Literary Criticism']

In [34]:
flist[1][1][1].keys()

[u'publisher',
 u'description',
 u'language',
 u'previewLink',
 u'readingModes',
 u'allowAnonLogging',
 u'publishedDate',
 u'printType',
 u'pageCount',
 u'maturityRating',
 u'contentVersion',
 u'industryIdentifiers',
 u'authors',
 u'title',
 u'imageLinks',
 u'infoLink',
 u'categories',
 u'canonicalVolumeLink']

In [22]:
pbooks=[]#load_obj('data/procbooksIm')
t0 = time.time()
try:
    p = Pool(10)
    books=p.map(movet, flist)
    #p.join()
    p.terminate()
    p.close()
    print "parallel end"
    for b in books:
        k=b[1][b[1].rindex('/')+1:-5]
        pbooks[k]=b[0]
        prockeys.append(k)
    save_obj(prockeys,'data/prockeysIm')
    save_obj(pbooks,'data/procbooksIm')
except:
    p.terminate()
    p.close()
t1=time.time()
total = t1-t0
print('time: ',total)

mv: cannot stat '/tmp/Aaron Allston - Star Wars Fate Of The Jedi - Outcast.epub': No such file or directory
parallel end
('time: ', 62.64743900299072)


In [66]:
print len(books),len(prockeys)

5 0


In [67]:
print "R: ",len(list (filter(lambda x:x[0]  =="R",books)))
print "W: ",len(list (filter(lambda x:x[0]  =="W",books)))
print "S: ",len(list (filter(lambda x:x[0]  =="S",books)))
print "F: ",len(list (filter(lambda x:x[0]  =="F",books)))
print "E: ",len(list (filter(lambda x:x[0]  =="E",books)))
print "A: ",len(list (filter(lambda x:x[0]  =="A",books)))

R:  0
W:  0
S:  4
F:  0
E:  1
A:  0


In [8]:
titles.keys()

['Han Solo Book 1 - The Paradise  - A. C. Crispin',
 'Han Solo Book 3 - Rebel Dawn - A. C. Crispin',
 'Angels and Insects - A. S. Byatt',
 'Possession - A. S. Byatt',
 'Slan - A. E. Van Vogt',
 'The Long War 01 - The Black Gua - A. J. Smith',
 'Babel Tower - A. S. Byatt',
 'The World of Null-A - A. E. Van Vogt',
 "The Children's Book - A. S. Byatt",
 'A Whistling Woman - A. S. Byatt',
 'N',
 'Skin Game_ A Novel of the Dresd - Jim Butcher',
 "Djinn in the Nightingale's Eye - A. S. Byatt",
 "The Biographer's Tale - A. S. Byatt",
 'Ragnarok_ The End of the Gods - A. S. Byatt',
 'Elementals - A. S. Byatt',
 'Han Solo Book 2 - The Hutt Gamb - A. C. Crispin']